In [1]:
from __future__ import print_function
import torch # A GPU-Ready Tensor Library for deep learning
import torch.nn as nn # These are the basic building blocks for graphs
import torch.nn.functional as F # torch lib for Convolution functions
import torch.optim as optim  # torch.optim is a package implementing various optimization algorithms. 
from torchvision import datasets, transforms #The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.

In [2]:
class Net(nn.Module):  # Net class inheriting nn.Module which is Base class for all neural network modules.
    def __init__(self):
        super(Net, self).__init__()                        #    input shape    output shape      RF
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)        #  28 x 28 x 1  - 28 x 28 x 32        3     # Convolution layer with i/p channels=2, 0/p channels=32, kernel size- 3 x 3. Padding =1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)       #  28 x 28 x 32 - 28 x 28 x 64        5 
        self.pool1 = nn.MaxPool2d(2, 2)                    #  28 x 28 x 64 - 14 x 14 x 64        10 
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)      #  14 x 14 x 64 - 14 x 14 x 128       12 
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)     #  14 x 14 x 128 - 14 x 14 x 256      14
        self.pool2 = nn.MaxPool2d(2, 2)                    #  14 x 14 x 256 - 7 x 7 x 256        28
        self.conv5 = nn.Conv2d(256, 512, 3)                #  7 x 7 x 256 -   5 x 5 x 512        30
        self.conv6 = nn.Conv2d(512, 1024, 3)               #  5 x 5 x 512 -   3 x 3 x 512        32
        self.conv7 = nn.Conv2d(1024, 10, 3)                #  3 x 3 x 512 -   1 x 1 x 512        34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))  #  i/p --> conv1--> relu --> conv2 ---> relu -> x
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))  # x ---> conv3 ---> relu ---> conv4 --> relu ---> x
        x = F.relu(self.conv6(F.relu(self.conv5(x))))   # x ---> conv5 --> relu --> conv6 ---> relu ---> x
        x = F.relu(self.conv7(x))  # x --> conv7 --> relu --> x
        x = x.view(-1, 10)  # reshaping for 10 classes
        return F.log_softmax(x)  #  applying softmax on o/p

In [3]:
!pip install torchsummary #   installing torchsummary library
from torchsummary import summary # summary provides the visualization of model
use_cuda = torch.cuda.is_available() # checking availability of CUDA
device = torch.device("cuda" if use_cuda else "cpu")  # check gpu availability and set the device
model = Net().to(device) # load model to device
summary(model, input_size=(1, 28, 28))  # prints the model visualization.  Layer names, output shape and number of parameters

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 

<ipython-input-2-98102ba2721d>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:


torch.manual_seed(1) # Sets the seed for generating random numbers.  
batch_size = 128 # 128 images processesd at once in gpu

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}  # setting workers
train_loader = torch.utils.data.DataLoader(           #  getting MNIST dataset provided by datasets class dataloader  # train dataloader
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([ # list of transforms to compose. 
                        transforms.ToTensor(),   # Convert a PIL Image or numpy.ndarray to tensor.
                        transforms.Normalize((0.1307,), (0.3081,))  # Normalizes the images
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs) # makes batches of images with shuffling 
test_loader = torch.utils.data.DataLoader(   # train dataloader
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm  # for printing progress bar
def train(model, device, train_loader, optimizer, epoch):
    model.train()  # putting model in train mode
    pbar = tqdm(train_loader) # initializing progress bar
    for batch_idx, (data, target) in enumerate(pbar):   # iterating over batches
        data, target = data.to(device), target.to(device)  # moving data to device ( cpu/ gpu )
        optimizer.zero_grad() # clearing the gradients of optimizer
        output = model(data) # forward pass to model using data
        loss = F.nll_loss(output, target) # computing negative log loss on model output and target
        loss.backward() # propagate loss bacward
        optimizer.step() # updating weights 
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}') # progressbar updated


def test(model, device, test_loader):
    model.eval()  # putting model in eval mode
    test_loss = 0
    correct = 0
    with torch.no_grad(): # no gradients computed
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # moving data to device ( cpu/ gpu )
            output = model(data) # forward pass to model using data
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item() # adding the correct o/p items

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))      # accuracy = 100 x  # correct / # total 

In [ ]:

model = Net().to(device)  # # load model to device
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # stochastic gradient descent (optionally with momentum)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader) # call test func